![NVIDIA Logo](images/nvidia.png)

# Project: Automatic Email Responses

Eariler in the workshop you undertook several tasks to create synthetic customer emails that included a variety of specific details appropriate to a fictitious company and its industry. In this project, you will again use several prompt-engineered and fine-tuned models, including the sentiment analysis and extractive QA fine-tuned models you recently fine-tuned, but this time in the service of creating automatic responses to the synthetic customer emails you generated earlier in the workshop.

---

## Imports

In [ ]:
import json

from tqdm.notebook import tqdm

from llm_utils.nemo_service_models import NemoServiceBaseModel
from llm_utils.models import Models, LoraModels
from assessment import assess
from llm_utils.postprocessors import strip
from llm_utils.llm_functions import (
    autorespond_to_customer as solution_autorespond_to_customer,
    get_sentiment,
    make_llm_function
)

---

## Models

In [ ]:
Models.list_models()

In [ ]:
LoraModels.list_models()

---

## Project Main Objective

![Auto Respond](images/auto_respond.png)

The main objective of this exercise is to generate response emails of roughly 200 characters to the customer emails you synthetically generated earlier in the workshop.

In service of generating an appropriate response you will need to ascertain the following from the customer emails and then use them appropriately in the response email:

- Overall sentiment of the customer's email.
- The name of the person who sent the email.
- The product of focus in the customer's email.
- The store location where the customer purchased their product.

The result of your work should be a function `autorespond_to_customer` that expects a single `email` argument (a customer_email) for which it generates and returns an appropriate response.

---

## Exercise Example

Below is an example of the kind of email response your `autorespond_to_customer` function should generate. Notice especially how the response takes into account the sentiment of the customer email (by either thanking the customer or apologizing to them) and how its response includes the sender name, product, and store location details from the cutomer email.

First we'll provide 2 synthetic customer emails to generate auto responses for.

### Negative Customer Email

In [ ]:
negative_bike_seat_berkeley_josh = """
Heyo,

I recently got a SuperSeater bike seat for my toddler from your store in Berkeley and one of the seatbelt straps \
appears to be frayed pretty bad and I think it may have come that way. One of my neighbors actually pointed it \
out to me and they mentioned it might be a safety issue. I'm wondering if it's something you can repair, \
or if you can replace the bike seat for me so I can feel safe hauling my kid around? This seems pretty \
dangerous right now.

Best,
Josh
"""

In [ ]:
response_to_negative_bike_seat_berkeley_josh = solution_autorespond_to_customer(negative_bike_seat_berkeley_josh)

In [ ]:
print(response_to_negative_bike_seat_berkeley_josh)

### Positive Customer Email

In [ ]:
positive_cruiser_oceanside_marcello = """
Good Day,

I've been riding my new Starlight Cruiser for a couple weeks now and just want to report that I've \
been having a blast. I've been spending so much less time in my car and all I can say is \
I can't believe I waited so long! I'd like to chat with someone at your store in \
Oceanside about setting up regular tune-up appointments. Can you help me out with that?

Best,
Marcello
"""

In [ ]:
response_to_positive_cruiser_oceanside_marcello = solution_autorespond_to_customer(positive_cruiser_oceanside_marcello)

In [ ]:
print(response_to_positive_cruiser_oceanside_marcello)

---

## Project Components

![Auto Details](images/auto_details.png)

Your `autorespond_to_customer` function will be performing several specific tasks:
- Getting the sentiment of the customer email.
- Extracting the customer name from the email.
- Extracting the product in question from the email.
- Exracting the location where the product in question was purchased from the email.
- Generating a respone email using all of the pertinent details above.

With that in mind, we've provided the definition of `autorespond_to_customer`, and it will be your task to implement each of its component LLM function parts.

In [ ]:
company = 'StarBikes'
def autorespond_to_customer(email):
    sentiment = get_sentiment(email, tokens_to_generate=1)
    name = extract_name(email)
    product = extract_product(email)
    location = extract_location(email)

    response = generate_customer_response_email(company, name, sentiment, product, location)
    return response

---

## Sentiment Analysis

Earlier in the workshop, you created the `get_sentiment` LLM function, which uses a few-shot prompt engineered GPT8B model. We've imported it for your use.

In [ ]:
get_sentiment('I am happy', tokens_to_generate=1)

---

## Entity Extraction

For the various entity extraction tasks you will be using the LoRA fine-tuned GPT8B model for extractive QA tasks that you created in the previous notebook.

In [ ]:
extractor_model = NemoServiceBaseModel(LoraModels.gpt8b.value, customization_id='ebd552dc-a050-4987-afca-9136d45fbad1')

As you'll recall, `extractor_model` was fine-tuned using the following prompt template.

In [ ]:
def extract_template(text, question):
    return f'{text}\n{question} answer:'

You will be working with this model, in addition to creating appropriate prompt templates and postprocessors, to create 3 LLM functions: `extract_name`, `extract_product`, and `extract_location`.

### Extract Name

In [ ]:
def extract_name_template(email):
    return 'name'

In [ ]:
def extract_name_postprocessor(name):
    return name

In [ ]:
extract_name = make_llm_function(extractor_model, extract_name_template, postprocessor=extract_name_postprocessor)

### Extract Product

In [ ]:
def extract_product_template(email):
    return 'product'

In [ ]:
def extract_product_postprocessor(product):
    return product

In [ ]:
extract_product = make_llm_function(extractor_model, extract_product_template, postprocessor=extract_product_postprocessor)

### Extract Location

In [ ]:
def extract_location_template(email):
    return 'location'

In [ ]:
def extract_location_postprocessor(location):
    return location

In [ ]:
extract_location = make_llm_function(extractor_model, extract_location_template, postprocessor=extract_location_postprocessor)

---

## Response Email Generation

After analyzing the sentiment of the customer email and extracting all relevant details, you'll need to pass them to an LLM function you will create to generate the response to the customer. For this LLM function you will be using GPT43B with the provided prompt template, which you will need to complete.

In [ ]:
email_response_model = NemoServiceBaseModel(Models.gpt43b.value)

In [ ]:
def customer_response_email_prompt_template(company_name, recipient_name, sentiment, product, store_location):
    return 'email'

In [ ]:
generate_customer_response_email = make_llm_function(email_response_model, customer_response_email_prompt_template)

---

## Load Emails

Before beginning your work, load the synthetic customer emails from the previous notebook. We've provided our solution emails for your use here.

In [ ]:
with open('data/solution_emails.json', 'r') as f:
    customer_emails = json.load(f)

In [ ]:
len(customer_emails)

In [ ]:
customer_emails[0]

---

## Company Name

We will again use the fictitious bike company StarBikes.

In [ ]:
company = 'StarBikes'

---

## Assessing Your Work

If you are successfully able to implement a working `autorespond_to_customer` function you will earn a certificate of competency for the workshop. In order to assess your work, you will be passing your `autorespond_to_customer` function to our provided `assess` function.

Behind the scenes, `assess` will pass `autorespond_to_customer` 3 customer emails and then check that your response matches the specifications we've described in this notebook. In order to pass the assessment, your responses will need to contain fewer than 3 mistakes.

Since we've already implemented the scaffolding for `autorespond_to_customer` we can try the assessment out now, although it is definitely going to fail.

In [ ]:
assess(autorespond_to_customer)

### Earning a Workshop Certificate

Once you have successfully passed the assessment, see the instructions at the bottom of the notebook for how to get credit for your work and generate a certificate of competency for the workshop.

---

## Begin Your Work

If you're up for a big challenge, you can jump right in: we've provided you with everything you need to complete the assessment.

If you'd like additional support, expand the _Exercise Walkthrough_ section below to work through the assessment step by step.

### Your Work Here

---

# Exercise Walkthrough

Let's look again at the `autorespond_to_customer` function we need to make functional.

In [ ]:
def autorespond_to_customer(email):
    sentiment = get_sentiment(email, tokens_to_generate=1)
    name = extract_name(email)
    product = extract_product(email)
    location = extract_location(email)

    response = generate_customer_response_email(company, name, sentiment, product, location)
    return response

With this function definition as our guide, we will tackle each of the component LLM functions one at a time.

---

## Get Sentiment

![Sentiment LLM function](images/sentiment_llm_function.png)

You already implemented the LLM function `get_sentiment` and we have imported it for you, so all we need to do is confirm that it is working as expected by trying it out with a few of the `customer_emails` we loaded from file above.

In [ ]:
for customer_email in customer_emails[:5]:
    print(get_sentiment(customer_email, tokens_to_generate=1))

---

## Entity Extraction

For the entity extraction tasks you will be using the GPT8B model you LoRA fine-tuned for extractive question answering in the previous notebook, which we instantiate for you here. We've also provided the prompt template associated with the extractive QA task.

In [ ]:
extractor_model = NemoServiceBaseModel(LoraModels.gpt8b.value, customization_id='ebd552dc-a050-4987-afca-9136d45fbad1')

In [ ]:
def extract_template(text, question):
    return f'{text}\n{question} answer:'

## Entity Extraction Example

Up to now you've been performing extractive QA with the SQuAD dataset. Let's take a look at how we might use the model to extract details of interest to us, in this case a store location, from a synthetic customer email.

In [ ]:
text = """
Heyo,

I recently got a SuperSeater bike seat for my toddler from your store in Berkeley and one of the seatbelt straps \
appears to be frayed pretty bad and I think it may have come that way. One of my neighbors actually pointed it \
out to me and they mentioned it might be a safety issue. I'm wondering if it's something you can repair, \
or if you can replace the bike seat for me so I can feel safe hauling my kid around? This seems pretty \
dangerous right now.

Best,
Josh
"""

In [ ]:
extract_question = 'Where is the store located?'

In [ ]:
prompt = extract_template(text, extract_question)

In [ ]:
print(prompt)

In [ ]:
extractor_model.generate(prompt).strip()

At a glance at least, it looks like the model you fine-tuned might well be up for the currect extractive QA tasks you would like to address.

---

## Name Extractor

Since we would like to reuse the `extractor_model` for 3 different extraction tasks (name, product, and location) and create 3 different LLM functions we will need for each of the 3 extraction tasks to create an LLM function using the model that has an appropriate prompt template, and optionally, a post-processor.

Let's begin with name extraction. We will provide the entire process by which we created this LLM function before giving you a chance to build your own extraction LLM functions for products and locations.

---

## Name Extraction Prompt Template

To create a name extraction prompt template we are going to reuse the prompt template that the extraction model expects, shown here.

In [ ]:
def extract_template(text, question):
    return f'{text}\n{question} answer:'

Knowing that we want to extract the name of who sent a given email, we can hardcode in a question to extract this name, leaving the rest of the expected prompt template intact. Below is a prompt template we prompt engineered for this task.

In [ ]:
def extract_name_template(text):
    return f'Email: {text}\nWhat is the name of the person, if any, that sent the email? answer:'

---

## Name Extraction LLM Function

![Extract name](images/extract_name.png)

Now with `extractor_model` and `extract_name_template` we can create an LLM function `extract_name`.

In [ ]:
extract_name = make_llm_function(extractor_model, extract_name_template)

Let's try it out on some customer emails.

In [ ]:
for customer_email in customer_emails[:15]:
    response = extract_name(customer_email)
    print(response)

These look pretty good except that there appears to be white space that needs stripping, and, in a couple instances we got back the the name `'Hi'` or `'Hi Starbikes'` which are clearly not acutal names. Let's investigate further by looking at the emails when we get either of these responses.

In [ ]:
for customer_email in customer_emails[:15]:
    response = extract_name(customer_email)
    if 'hi' in response.lower() or 'StarBikes' in response:
        print(response)
        print(customer_email+'\n')

In two of the cases, it looks like the customer email doesn't actually include a name. In these scenarios it makes sense the model was unable to extract a customer name.

In the 3rd case we can only guess why the model responded with `'Hi StarBikes'` instead of `Virginia`, but one guess is that it thought the former more of a human full name compared to the latter which might be understood as the name of a state.

Observing that our model does a good job at this task, but not a perfect one, we will want to take care in postprocessing to handle scenarios where we don't get back a customer name.

---

## Name Extraction Postprocessor

Let's create a postprocessor function to address the issue above. The following `get_safe_name` function strips white space but also checks to see if a non-sensical name like `'hi'` is being extracted, in which case it returns a default value `'a customer who didn\'t give us their name'`. There are some additional non-sensical names included in the `unsafe` set that we discovered working more in depth with this model, so we've gone ahead and provided them for you here.

In [ ]:
def get_safe_name(name):
    unsafe = {'no', 'i', 'hello', 'hi', 'greetings', 'starbikes', 'dear'}
    name_words = name.strip().lower().split()
    if any(word in unsafe for word in name_words):
        return 'a customer who didn\'t give us their name'
    else:
        return name.strip()

---

## Name Extraction LLM Function With Postproceesor

Let's recreate the `extract_name` LLM function, but this time using the `get_safe_name` postprocessor.

In [ ]:
extract_name = make_llm_function(extractor_model, 
                                 extract_name_template, 
                                 postprocessor=get_safe_name)

And now let's try the updated LLM function on several emails.

In [ ]:
for customer_email in customer_emails[:15]:
    print(extract_name(customer_email))

---

## Product Extractor

Using a similar approach, create the `extract_product` LLM function we need for use in our `autorespond_to_customer` email. After observing an initial implementation, consider adding a postprocessor to handle any unexpected responses by returning a default value like the string `'product'`.

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

## Solution

### Product Extractor Template

Knowing that we want to extract the name of who sent a given email, we can hardcode in a question to extract this name, leaving the rest of the expected prompt template intact.

In [ ]:
def extract_product_template(text):
    return f'{text}\nWhat StarBikes product is this person writing about? answer:'

### Product Extractor Postprocessor

In [ ]:
def get_safe_product(product):
    return 'product' if 'starbikes' in product.strip().lower() else product.strip()

### Product Extractor LLM Function

In [ ]:
extract_product = make_llm_function(extractor_model, 
                                    extract_product_template, 
                                    postprocessor=get_safe_product)

In [ ]:
for customer_email in customer_emails[:15]:
    response = extract_product(customer_email)
    print(response)
    if 'StarBikes' in response:
        print(customer_email+'\n')

---

## Location Extractor

![Extract location](images/extract_location.png)

Now create the `extract_location` LLM function. After observing an initial implementation, consider adding a postprocessor to handle any unexpected responses by returning a default value like the string `"their store's location"`.

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

## Solution

### Location Extractor Template

In [ ]:
def extract_location_template(text):
    return f'{text}\nWhere was the store located? answer:'

### Location Extractor Postprocessor

In [ ]:
def get_safe_location(location):
    return 'their store\'s location' if location.strip().lower() == 'no' else location.strip()

### Location Extractor LLM Function

In [ ]:
extract_location = make_llm_function(extractor_model, 
                                     extract_location_template, 
                                     postprocessor=get_safe_location)

In [ ]:
for customer_email in customer_emails[:15]:
    print(extract_location(customer_email))

---

## Write Response Email LLM Function

![Write response](images/write_response.png)

Now it's time to put your prompt engineering skills to the test by creating a `generate_customer_response_email` LLM function. Your function should accept as arguments the sentiment of the customer emails as well as the 3 extracted details (name, product and location) and should respond appropriately to the sentiment of the customer email, reusing the customer name, product and location used in the cutomer email.

We provided an [example of this functionality earlier in the notebook](#Exercise-Example) if you'd like to review it now.

Provided for you below is the model for you to use and the scaffolding for the prompt template you should use. As a reminder, `company_name` was defined eariler in the notebook and should be available for your use here.

Feel free to check out our *Solution* below if you get stuck.

### Your Work Here

In [ ]:
email_response_model = NemoServiceBaseModel(Models.gpt43b.value)

In [ ]:
def customer_response_email_prompt_template(company_name, recipient_name, sentiment, product, store_location):
    return 'email'

In [ ]:
generate_customer_response_email = make_llm_function(email_response_model, customer_response_email_prompt_template)

## Solution

### Customer Response Email Prompt Template

We iteratively engineered the following prompt template for the customer response email.

In [ ]:
def customer_response_email_prompt_template(company_name, recipient_name, sentiment, product, store_location):
    return f"""\
Write a customer response email of 200 words.

Context: {recipient_name} just sent an email expressing a {sentiment} sentiment about our company {company_name}. \
Their email was about a {product} they purchased in {store_location}. We want to send a response email emphathizing with \
their experience and if appropriate, telling them that someone from {store_location} will be contacting them soon.

Instructions: Write a reply email using the following steps:

1) Greet {recipient_name} professionally by their name. If their name is 'the customer' address them as "Dear Customer".

2) Depending on the sentiment expressed in their email (stated above), empathize as a friend would about their experience with their {product}.

3) Tell the customer that an employee from our store in {store_location} will contact them soon \
to followup with them in more detail. NEVER ask the customer to contact us. NEVER ask the customer for thier contact information.

4) Write a professional closing signed by "{company_name} Customer Support Team"
"""

### Customer Response Email LLM Function

Using the above model, prompt template, and `strip` postprocessor, we created the following LLM function.

In [ ]:
generate_customer_response_email = make_llm_function(email_response_model, 
                                                     customer_response_email_prompt_template, 
                                                     postprocessor=strip) 

Let's try it out.

In [ ]:
customer_response_email = generate_customer_response_email('StarBikes', 'Stella', 'negative', 'Cruiser', 'Oakland')
print(customer_response_email)

---

## Autorespond to Customer Emails

![Auto Details](images/auto_details.png)

Now that we've created all the component LLM functions for `autorespond_to_customer` let's try it out on a few customer emails.

First we'll provide the `autorespond_to_customer` definition from earlier in the notebook.

In [ ]:
company = 'StarBikes'
def autorespond_to_customer(email):
    sentiment = get_sentiment(email, tokens_to_generate=1)
    name = extract_name(email)
    product = extract_product(email)
    location = extract_location(email)

    response = generate_customer_response_email(company, name, sentiment, product, location)
    return response

Now we'll try it on a few customer emails.

In [ ]:
for customer_email in tqdm(customer_emails[:4]):
    customer_response_email = autorespond_to_customer(customer_email)
    print(customer_email+'\n\n')
    print(customer_response_email+'\n---\n')

----

# Workshop Final Assessment and Certificate

If you've successfully been able to implement a working `autorespond_to_customer` function you will earn a certificate of competency for the workshop. In order to assess your work, you pass your `autorespond_to_customer` function to our provided `assess` function.

`assess` will pass `autorespond_to_customer` 3 customer emails and then check that your response matches the specifications we've described in this notebook. In order to pass the assessment, your responses will need to contain fewer than 3 mistakes.

In [ ]:
assess(autorespond_to_customer)

---

## Get Credit for Your Work

If you've made it this far, congratulations! You did a lot of hard work today and your efforts have paid off.

Assuming you ran `assess(autorespond_to_customer)` above and got a message saying you passed, you can get a certificate of competency for the course.

In your web browser, return to the page where you launched this interactive environment and click the check-mark `ASSESS TASK` button. After a few seconds you will get a congratulatory message with instructions for receiving your certificate in the course.

![assess](images/assess.png)